## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
%matplotlib inline

In [ ]:
path = 'drive/MyDrive/ADM/movielens/'

## Content Embeddings

In [ ]:
contentEmbeddings = pd.read_pickle(path + 'ae_emb.pkl')
contentEmbeddings = pd.DataFrame(contentEmbeddings)
contentEmbeddings.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.000000,0.0,0.0,0.766454,0.0,1.055996,0.0,1.186909,1.041592,0.027471,0.038791,0.0,0.0,1.052784,0.0,1.051433,0.000000,0.0,0.000000,0.0,0.987328,0.676191,0.0,0.0,0.924645,0.104829,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.916857,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.004701,0.0,1.406821,0.0,0.000000,0.0,0.000000,0.0,0.0,0.645232,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311465,0.924861,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.237403,0.0,1.063039,0.0,1.176590,0.908302,0.466266,0.000000,0.0,0.0,1.013608,0.0,1.124436,0.375900,0.0,0.418915,0.0,1.084167,0.886606,0.0,0.0,1.016657,0.455792,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.970824,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.070994,0.0,0.0,0.000000,0.0,0.937433,0.0,0.850563,0.0,0.000000,0.0,0.000000,0.0,0.0,0.450283,0.0,0.164884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.676915,0.970927,0.0,0.0,0.0,0.0,0.0,0.171496,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.279502,0.0,1.024473,0.0,1.124208,0.986693,0.234315,0.133252,0.0,0.0,1.013220,0.0,1.081190,0.529603,0.0,0.099515,0.0,1.053401,0.795152,0.0,0.0,1.002760,0.166486,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.823611,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.183999,0.0,0.0,0.000000,0.0,0.955058,0.0,0.426210,0.0,0.000000,0.0,0.000000,0.0,0.0,0.742623,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.380977,0.870311,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.850509,0.0,0.962969,0.777785,0.144186,0.000000,0.0,0.0,0.788484,0.0,0.966232,0.000000,0.0,0.102190,0.0,0.900891,0.671248,0.0,0.0,0.768499,0.259112,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.815117,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.848825,0.0,0.352404,0.0,0.000000,0.0,0.000000,0.0,0.0,0.078862,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.293195,0.776205,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.188846,0.0,0.0,0.000000,0.0,0.847974,0.0,0.970052,0.775281,0.397921,0.000000,0.0,0.0,0.811454,0.0,1.007620,0.403544,0.0,0.292478,0.0,0.988903,0.714996,0.0,0.0,0.822572,0.448337,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.893452,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.049017,0.085540,0.0,0.0,0.212263,0.0,0.873942,0.0,0.621830,0.0,0.000000,0.0,0.000000,0.0,0.0,0.137300,0.0,0.042929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415929,0.837247,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5,0.0,0.015807,0.0,0.0,0.000000,0.0,1.004330,0.0,1.185809,0.921709,0.523194,0.000000,0.0,0.0,0.875924,0.0,1.075781,0.000000,0.0,0.113607,0.0,1.110519,0.869102,0.0,0.0,0.949499,0.529286,0.0,0.0,0.0,0.005829,0.0,0.0,0.0,0.963714,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.048061,0.0,0.334186,0.0,0.000000,0.0,0.060267,0.0,0.0,0.050228,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.276908,0.969913,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.000000,0.0,1.089715,0.0,1.244597,0.984263,0.262343,0.000000,0.0,0.0,1.037917,0.0,1.157838,0.135566,0.0,0.384818,0.0,1.013804,0.841912,0.0,0.0,0.988609,0.576589,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.939595,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.127462,0.000000,0.0,0.0,0.000000,0.0,1.005853,0.0,0.420612,0.0,0.000000,0.0,0.289476,0.0,0.0,0.426743,0.0,0.144481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.787910,0.991015,0.0,0.0,0.0,0.0,0.0,0.301240,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.179662,0.0,0.901787,0.0,0.997542,0.950469,0.213240,0.000000,0.0,0.0,0.756171,0.0,1.033867,0.033743,0.0,0.112966,0.0,1.015272,0.651100,0.0,0.0,0.849379,0.511919,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.910792,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.424200,0.0,0.0,0.062092,0.0,0.906226,0.0,0.660756,0.0,0.018403,0.0,0.000000,0.0,0.0,0.000000,0.0,0.142390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.401469,0.781461,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

## Collaborative Embeddings

In [ ]:
collaborativeEmbeddings = pd.read_pickle(path + 'movie_embeddings_1.pkl')
collaborativeEmbeddings.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.029365,-0.029352,-0.031192,-0.011644,-0.017321,0.027635,-0.048266,-0.014380,0.024477,0.048038,0.015033,-0.032807,-0.015022,-0.041547,0.009819,0.026248,0.022276,0.005479,0.039941,-0.029774,-0.048435,0.019073,0.013811,-0.019916,-0.018471,-0.033017,0.023823,-0.019553,0.014853,0.019132,-0.029028,-0.020750,-0.008621,-0.044085,0.042407,-0.038400,-0.033289,-0.031650,0.045687,0.005980,...,0.013923,0.036151,0.010546,0.036429,-0.002848,-0.010380,0.003970,0.021355,0.001329,0.027800,0.030653,-0.022809,-0.049498,0.004946,-0.024860,-0.046659,-0.041230,0.022374,-0.038307,-0.004580,0.020825,-0.025359,-0.033808,-0.001806,0.043326,0.023161,-0.041311,-0.040592,0.002877,-0.001685,-0.049403,-0.029768,-0.023093,-0.001951,-0.024746,0.039837,-0.048239,0.008592,-0.008328,-0.003379
1,0.017979,0.021861,-0.020371,0.034225,-0.018364,-0.015082,0.013061,-0.022779,0.046697,0.033575,-0.011691,0.015646,-0.025193,0.018183,0.039227,-0.013348,-0.023918,-0.048387,0.006869,0.041083,-0.033371,0.023477,-0.012268,-0.027155,0.044244,-0.035799,-0.024155,-0.046429,-0.040337,0.010201,-0.021638,0.039827,0.009126,0.017741,0.029557,-0.015680,-0.032615,-0.019456,0.033508,0.047420,...,-0.034168,0.014224,0.002301,-0.014785,-0.045766,0.038498,-0.047530,0.015993,-0.017447,-0.017800,0.024391,-0.011798,-0.048228,-0.035366,-0.015377,0.033622,0.001454,-0.000187,0.007211,0.033817,0.045169,-0.036393,-0.046345,0.024731,-0.011266,0.042422,-0.019143,-0.019108,0.000306,-0.013850,0.043570,0.009858,-0.012329,0.007445,0.046385,0.023931,0.034497,0.043097,0.008297,0.010601
2,0.009159,0.026536,0.034986,-0.017784,0.025985,-0.022536,0.033272,-0.035988,-0.044241,-0.045214,0.044102,0.016914,-0.023849,0.045103,0.043760,0.018727,-0.003763,-0.027409,-0.029352,0.044595,-0.009993,-0.037373,-0.003211,-0.012726,-0.043663,-0.015853,0.027847,0.027257,-0.025109,0.037326,-0.019451,0.015305,-0.015633,0.022598,-0.000881,0.002354,0.001324,-0.014823,0.024128,0.024406,...,0.022164,-0.008186,0.006104,0.022490,-0.004039,-0.040238,-0.036687,-0.047778,0.009596,0.003907,0.003069,-0.031239,-0.023164,-0.008821,0.014319,0.021868,0.019241,0.012407,-0.039561,0.016320,-0.021335,0.045398,0.021513,0.022858,0.028230,0.047873,-0.007737,-0.020242,-0.001782,0.038006,0.031825,0.043369,0.021687,-0.015399,0.013869,0.042577,0.033876,-0.031735,-0.019296,0.015888
3,0.010538,0.013007,0.012423,-0.035767,-0.004829,-0.032602,-0.031653,0.000089,0.042884,0.038748,0.035982,0.011026,0.027960,-0.019788,-0.004395,-0.025756,-0.009497,-0.014928,-0.031775,-0.042622,0.005310,0.049605,-0.044716,-0.049287,-0.002071,0.014542,-0.025427,0.023006,0.002691,0.040283,0.004134,-0.006636,-0.009559,-0.030185,-0.023359,-0.030729,-0.019825,-0.043477,0.026290,0.010601,...,0.008986,0.023580,0.011035,0.037979,-0.031930,0.040118,0.040437,-0.000179,0.013078,0.049546,-0.006609,0.009280,-0.019096,-0.002418,-0.005310,0.023063,0.022251,0.002470,-0.016337,-0.024940,0.035138,-0.004615,-0.017182,0.039303,-0.000426,0.027448,0.023604,-0.021374,-0.010559,-0.037031,-0.044096,-0.014027,0.028190,0.044919,0.029261,-0.041752,0.020794,0.029635,0.026661,-0.031745
4,0.010987,0.012467,0.028735,0.024920,0.009714,0.035787,-0.031599,-0.026101,0.034219,-0.031272,-0.004391,-0.016533,-0.015860,-0.033504,-0.034578,-0.004860,-0.043255,0.023480,-0.034983,0.049036,-0.019270,-0.016232,-0.028217,-0.028970,-0.011084,0.002896,-0.004946,-0.026965,0.013688,0.014753,-0.009338,-0.043756,-0.009231,-0.011804,0.008422,0.000907,-0.033795,0.045552,0.015625,-0.045685,...,-0.009139,-0.027125,-0.036395,0.042138,0.018950,0.040599,0.022116,-0.027499,0.031966,-0.016910,0.006436,0.032476,-0.011495,-0.021411,-0.017820,0.002852,-0.022243,-0.044065,-0.022341,0.023003,-0.019658,-0.012352,-0.031603,0.016154,-0.045130,-0.046103,0.010301,-0.004774,-0.032249,0.010646,0.044860,0.016238,-0.01

## Formatting Movie Lookup Data - Preprocessing

In [ ]:
ratings_data = pd.read_csv(path + 'ratings.csv')

In [ ]:
# movie to sequential non-breaking index
movieID_unique = ratings_data.movieId.unique()
movie2index = {o:i for i,o in enumerate(movieID_unique)}

In [ ]:
movies_data = pd.read_csv(path + 'movies.csv')
print("{} unique movies in movies.csv".format(len(movies_data.movieId.unique())))

ratings_data = pd.read_csv(path + 'ratings.csv')
print("{} unique movies in ratings.csv".format(len(ratings_data.movieId.unique())))

movies_data = pd.merge(movies_data, ratings_data, on="movieId", how="inner")
movies_data.movieId = movies_data.movieId.apply(lambda x: movie2index[x])

# Get popularity
popularity = pd.DataFrame(movies_data[['userId', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['count']))
popularity.reset_index(inplace=True)
popularity.columns = ['title', 'movieId', 'ratings_data_count']
popularity.sort_values('ratings_data_count', ascending=False, inplace=True)
movies_data = pd.merge(popularity[['movieId', 'ratings_data_count']], movies_data, on='movieId')
movies_data.reset_index(inplace=True)

# Get average ratings
average_ratings = pd.DataFrame(movies_data[['rating', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['mean']))
average_ratings.reset_index(inplace=True)
average_ratings.columns = ['title', 'movieId', 'avg_rating']
movies_data = pd.merge(average_ratings[['movieId', 'avg_rating']], movies_data, on='movieId')
movies_data.reset_index(inplace=True)


movies_data = movies_data[['movieId', 'title', 'genres', 'ratings_data_count', 'avg_rating']]
movies_data.drop_duplicates(inplace=True)
print("{} unique movies in embeddings".format(len(movies_data.movieId.unique())))
movies_data.set_index('movieId', inplace=True, drop=True)
movies_data.sort_index(ascending=True, inplace=True)
print(movies_data.shape)
movies_data.head(5)

27278 unique movies in movies.csv
26744 unique movies in ratings.csv
26744 unique movies in embeddings
(26744, 4)


,title,genres,ratings_count,avg_rating
movieId,,,,
0,Jumanji (1995),Adventure|Children|Fantasy,22243,3.211977
1,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,8520,3.952230
2,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,44980,3.898055
3,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,43249,4.053493
4,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,47006,4.334372


In [ ]:
movies_data.to_csv(path + 'movie_demographics.csv')

In [ ]:
movies_data.query('title == "Zodiac (2007)"')

,title,genres,ratings_count,avg_rating
movieId,,,,
3995,Zodiac (2007),Crime|Drama|Thriller,3907,3.675454


## Recommendation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

class SimilarityPredictions(object):
    def __init__(self, embeddings, similarity_metric='cosine'):
        assert similarity_metric in ['cosine', 'euclidean'], "unsupported similarity metric."
        self.embeddings = embeddings
        self.IDs = embeddings.index.tolist()
        self.sim_metric = similarity_metric
        if similarity_metric == 'cosine':
            self.sim_matrix = self.calculate_cosine_sim_matrix()
        if similarity_metric == 'euclidean':
            self.sim_matrix = self.calculate_euclidean_dist_matrix()

    def calculate_cosine_sim_matrix(self):
        '''Calculates a cosine similarity matrix from the embeddings'''
        sim_matrix = pd.DataFrame(cosine_similarity(
            X = self.embeddings),
            index = self.IDs)
        sim_matrix.columns = self.IDs
        return sim_matrix

    def calculate_euclidean_dist_matrix(self):
        '''Calculates a euclidean distances matrix (sim_matrix) from the embeddings'''
        sim_matrix = pd.DataFrame(euclidean_distances(
            X = self.embeddings),
            index = self.IDs)
        sim_matrix.columns = self.IDs
        return sim_matrix

    def predict_sim_items(self, seed_item, n):
        '''Use the sim_matrix to return n most similar items.'''
        sim_items = pd.DataFrame(self.sim_matrix.loc[seed_item])
        sim_items.columns = ["similarity_score"]
        if self.sim_metric == 'cosine':
            sim_items = sim_items.sort_values('similarity_score', ascending = False)
        if self.sim_metric == 'euclidean':
            sim_items = sim_items.sort_values('similarity_score', ascending = True)
        sim_items = sim_items.head(n)
        sim_items.reset_index(inplace = True)
        sim_items = sim_items.rename(index = str, columns = {"index": "item_id"})
        return sim_items.to_dict()

In [ ]:
def movieID_lookup_by_title(movie_title):
  return movies_data[movies_data.title.str.contains(movie_title)]

In [ ]:
movieID_lookup_by_title('Notebook')

,title,genres,ratings_count,avg_rating
movieId,,,,
1764,"Notebook, The (2004)",Drama|Romance,3968,3.804057
15131,"Notebook, The (A nagy füzet) (2013)",Drama|War,10,3.700000


In [ ]:
# Example
zodiac = 3995
lotr = 131    # Lord of the rings

In [ ]:
def get_detailed_records(movie_id, embeddings, file_path):
    # Get similar movies
    sim_model = SimilarityPredictions(embeddings, similarity_metric = "cosine")
    output = sim_model.predict_sim_items(seed_item = movie_id, n = 20)
    sim_movies = pd.DataFrame(output)
    sim_movies.set_index('item_id', inplace = True)
    sim_df = pd.merge(movies_data, sim_movies, left_index = True, right_index = True)
    sim_df.sort_values('similarity_score', ascending = False, inplace = True)
    
    # Save records locally
    sim_df.head(20).to_csv(file_path, index = False, header = True)
    return sim_df.head(20)

In [ ]:
def get_ensemble_records(movie_id, contentEmbeddings, collaborativeEmbeddings, file_path):
    # Get similar movies via content
    sim_model_content = SimilarityPredictions(contentEmbeddings, similarity_metric = "cosine")
    content_output = sim_model_content.predict_sim_items(seed_item = movie_id, n = 26744)
    sim_movies = pd.DataFrame(content_output)
    sim_movies.set_index('item_id', inplace=True)
    sim_df_content = pd.merge(movies_data, sim_movies, left_index = True, right_index = True)
    sim_df_content.sort_values('similarity_score', ascending = False, inplace = True)
    sim_df_content = sim_df_content.rename(index = str, columns = {"similarity_score": "content_similarity_score"})

    # Get similar movies via collaborative
    sim_model_collab = SimilarityPredictions(collaborativeEmbeddings, similarity_metric = "cosine")
    collab_output = sim_model_collab.predict_sim_items(seed_item = movie_id, n = 26744)
    sim_movies = pd.DataFrame(collab_output)
    sim_movies.set_index('item_id', inplace = True)
    sim_df_collab = pd.merge(movies_data, sim_movies, left_index = True, right_index = True)
    sim_df_collab.sort_values('similarity_score', ascending = False, inplace = True)
    sim_df_collab = sim_df_collab.rename(index = str, columns = {"similarity_score": "collaborative_similarity_score"})

    # Ensemble results
    sim_df_average = pd.merge(sim_df_collab, pd.DataFrame(sim_df_content['content_similarity_score']), left_index = True, right_index = True)
    sim_df_average['average_similarity_score'] = (sim_df_average['content_similarity_score'] + sim_df_average['collaborative_similarity_score'])/2
    sim_df_average.sort_values('average_similarity_score', ascending=False, inplace=True)
    
    # Save records locally
    sim_df_average.head(20).to_csv(file_path, index=False, header=True)
    return sim_df_average.head(20)

### Collaborative Filtering

In [ ]:
notebook = 1764

In [ ]:
get_detailed_records(notebook, collaborativeEmbeddings, path + 'collab_recs_notebook.csv')

,title,genres,ratings_count,avg_rating,similarity_score
1764,"Notebook, The (2004)",Drama|Romance,3968,3.804057,1.000000
20925,Collision Course (1989),Action|Comedy|Thriller,2,1.250000,0.368735
18266,Moontide (1942),Drama|Film-Noir,6,3.500000,0.359472
4932,Bob le Flambeur (1955),Crime|Drama,340,3.898529,0.349049
9321,Fawlty Towers (1975-1979),Comedy,230,4.128261,0.347769
6101,Nanook of the North (1922),Documentary|Drama,150,3.733333,0.346238
2417,Local Hero (1983),Comedy,3194,3.966656,0.343560
9287,2 Days in New York (2012),Comedy,86,3.151163,0.337562
19436,The Mad Magician (1954),Horror|Mystery|Thriller,3,2.833333,0.327152
6726,Longtime Companion (1990),Drama,466,3.780043,0.324915


### Content Based Filtering

In [ ]:
get_detailed_records(notebook, contentEmbeddings, path + 'content_recs_notebook.csv')

,title,genres,ratings_count,avg_rating,similarity_score
1764,"Notebook, The (2004)",Drama|Romance,3968,3.804057,1.000000
528,Notting Hill (1999),Comedy|Romance,12588,3.453924,0.978334
599,Mystic Pizza (1988),Comedy|Drama|Romance,2000,3.312000,0.976591
2556,"Family Stone, The (2005)",Comedy|Drama|Romance,969,3.223942,0.975787
19881,"Rains Came, The (1939)",Adventure|Drama|Romance,4,3.000000,0.975587
4128,Mrs. Winterbourne (1996),Comedy|Romance,1172,3.055461,0.975418
6173,Lost in Austen (2008),Drama|Fantasy|Romance|Sci-Fi,125,3.664000,0.975268
7365,"Elementary Particles, The (Elementarteilchen) ...",Drama|Romance,64,3.562500,0.974241
7962,Persuasion (2007),Drama|Romance,214,3.855140,0.973877
24203,"Human Comedy, The (1943)",Drama,7,3.500000,0.973088


### Hybrid Recommender

In [ ]:
get_ensemble_records(notebook, contentEmbeddings, collaborativeEmbeddings, path + 'ensembled_recs_notebook.csv')

,title,genres,ratings_count,avg_rating,collaborative_similarity_score,content_similarity_score,average_similarity_score
1764,"Notebook, The (2004)",Drama|Romance,3968,3.804057,1.000000,1.000000,1.000000
4932,Bob le Flambeur (1955),Crime|Drama,340,3.898529,0.349049,0.969552,0.659300
9321,Fawlty Towers (1975-1979),Comedy,230,4.128261,0.347769,0.930327,0.639048
9436,Flipped (2010),Comedy|Drama|Romance,156,3.717949,0.323495,0.931158,0.627327
18266,Moontide (1942),Drama|Film-Noir,6,3.500000,0.359472,0.891744,0.625608
6101,Nanook of the North (1922),Documentary|Drama,150,3.733333,0.346238,0.891713,0.618975
3898,Devil (2010),Horror|Mystery|Thriller,383,2.950392,0.309428,0.917075,0.613252
6726,Longtime Companion (1990),Drama,466,3.780043,0.324915,0.900093,0.612504
2417,Local Hero (1983),Comedy,3194,3.966656,0.343560,0.881083,0.612322
10327,Broken Blossoms or The Yellow Man and the Girl...,Drama|Romance,147,3.595238,0.297480,0.926454,0.611967
